In [ ]:
!pip install fastapi uvicorn pandas scikit-learn numpy pyngrok

In [ ]:
%%writefile app.py

from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

app = FastAPI(title="MNIST Decision Tree API")


train_df = pd.read_csv("mnist_train.csv")
test_df = pd.read_csv("mnist_test.csv")

X_train = train_df.drop("label", axis=1)
y_train = train_df["label"]

X_test = test_df.drop("label", axis=1)
y_test = test_df["label"]


model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

class MNISTInput(BaseModel):
    pixels: list

@app.get("/")
def home():
    return {"message": "MNIST Decision Tree API running 🚀"}

@app.get("/accuracy")
def get_accuracy():
    return {"accuracy": accuracy}

@app.post("/predict")
def predict_digit(data: MNISTInput):
    if len(data.pixels) != 784:
        return {"error": "Input must contain 784 pixel values"}

    sample = np.array(data.pixels).reshape(1, -1)
    pred = model.predict(sample)[0]
    return {"predicted_digit": int(pred)}


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import uvicorn
import threading

public_url = ngrok.connect(8000)
print("Public URL:", public_url)


threading.Thread(
    target=uvicorn.run,
    args=("app:app",),
    kwargs={"host": "0.0.0.0", "port": 8000},
    daemon=True
).start()


Public URL: NgrokTunnel: "https://shawnta-unsubdued-immovably.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [1943]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
